# Tutorial

This notebook will highlight key features of DataJoint pipelines and teach you to load and fetch data from custom tables.

In our experiment, we took 1-dimensional position measurements of mice using one of two cameras. Camera A had a sampling rate of 30 frames/sec and Camera B had a sampling rate of 45 frames/sec. We’re interested in (a) each subject’s average travel time and velocity, and (b) the relationship between subject age and velocity.

## Learning goals

By the end of this tutorial, you will understand…

- DataJoint table types (Manual, Lookup, Computed, Imported),
- Primary keys & secondary attributes in a relational database table,
- Automated data processing in DataJoint tables,
- Querying DataJoint tables.


## DataJoint Principles

We'll import datajoint and declare a schema with our username.


In [ ]:
import os
import datajoint as dj

schema = dj.Schema(f"{dj.config['database.user']}_ccn2023_") # "YOUR-USERNAM_ccn2023_"
HOME = os.getenv("HOME")
# schema.drop() # To start over, drop this schema


## Data
Please generate the position data running the cell below:

In [ ]:
import numpy as np

def simulate_kinematics_data(n=10):    
    for j in range(n):
        camera, fps_factor = ("A", 1) if j % 2 == 0 else ("B", 1.5)  # Alternate cams
        x = 0
        final = [x]
        while x < 50:  # Before position reaches 50, end of track
            x += (  # Increment X
                np.sign(
                    (np.random.randn() + 0.1)  # Randomly move, bias forward
                    * ((j + 1) / 2)  # Higher N younger subjects move faster
                )
                / fps_factor  # Slow down subjects with faster fps
            )
            final.append(x)  # Add to numpy array
        
        np.save(f"{HOME}/subject{j}_camera{camera}_kinematics.npy", final)

simulate_kinematics_data(n=10)

### Declaring tables

Next, we'll create a **Subject** table to store sex and date of birth information.


In [ ]:
@schema  # Decorator places the table within the schema
class Subject(dj.Manual):  # The table is a class that inherits from the Manual type
    definition = """          # Here, we define the table itself, with variable: data type
    subject_id: int           # Above the '---' separator is the primary key
    ---
    sex: enum('M', 'F', 'U')  # Sex and DOB are secondary attributes 
    dob: date
    """


To look at the table information, we can use any of the following:


In [ ]:
Subject()  # Show table and contents
# Subject.heading       # Show table definition


### Data Entry

We can manually enter data in many different formats (e.g., `list`, `dict`, `tuple`, CSV).

Here, we'll manually enter two subjects using lists that correspond to the fields above.


In [ ]:
Subject.insert([[0, "M", "2023-01-01"], [1, "F", "2023-01-02"]])


Confirming data entry...


In [ ]:
Subject()


### Data Queries

We can `fetch` one or more fields of a table as follows:


In [ ]:
data = Subject.fetch()
print("All data: ", data)


In [ ]:
sexes, dobs = Subject.fetch("sex", "dob")
print("Just sex: ", sexes)
print("Just DOB: ", dobs)


Note that DOB is in `datetime.date` format. While we can enter strings, they are formatted according to the table's datatype have the benefits of specificity over simple strings.


In [ ]:
from datetime import date

print(f"Age: {(date.today() - dobs[0]).days}")


### Table Types

Next, let's add tables for **Equipment** and **Recording** information.

The **Manual** `Subject` table above is designed for regular data entry. **Lookup**
tables are for reference information and permit default content. This is useful for
complex we wouldn't want to repeat in each row of another table, like parameter sets.


In [ ]:
@schema
class Equipment(dj.Lookup):
    definition = """
    equipment: varchar(8)
    ---
    sampling_rate: int  # frame per second [1/s]
    """
    contents = [("cameraA", 30), ("cameraB", 45)]


**Imported** and **Computed** tables are both associated with processing via their
`make` functions. Imported tables pull data from files, whereas Computed tables populate
from data already in the pipeline.


In [ ]:
@schema
class Recording(dj.Imported):
    definition = """               
    -> Subject                     # -> draw connections to other tables
    ---                            # These are foreign key references
    -> Equipment
    recording_file: varchar(64)
    position : longblob            # longblob can be used to store large arbitrary data
    """

    def make(self, key):
        from pathlib import Path
        import numpy as np

        subject_id = (Subject & key).fetch1("subject_id")  # fetch id from 'upstream'
        full_path = list(Path(f"{HOME}").glob(f"subject{subject_id}*npy"))[0].as_posix()
        relative_path = list(Path(f"{HOME}").glob(f"subject{subject_id}*npy"))[0].relative_to(f"{HOME}").as_posix() # look for the subj's file        
        
        equipment = relative_path.split("_")[1]  # find the camera id in the file name
        position = np.load(full_path)  # load position information

        self.insert1(
            {
                **key,
                "equipment": equipment,
                "position": position,
                "recording_file": relative_path,
            }
        )


To execute the `make` function, we run the `populate` method.

Note: populate will only operate once for each foreign key. To make adjustments and
rerun, use the `delete` method to clear current entries or the `drop` method the delete
the table entirely.


In [ ]:
Recording.populate()
Recording()


In the following **Computed** table, we'll process the position data with another `make`
function.


In [ ]:
@schema
class Kinematics(dj.Computed):
    definition = """
    -> Subject
    ---
    displacement: longblob
    travel_time: float
    average_velocity: float
    """

    def make(self, key):
        import numpy as np

        # This combines tables before fetching. See Operators below
        sampling_rate, position = (Recording * Equipment & key).fetch1(
            "sampling_rate", "position"
        )

        displacement = position[-1] - position[0]
        travel_time = len(position) / sampling_rate
        average_velocity = displacement / travel_time

        self.insert1(
            {
                **key,
                "displacement": displacement,
                "travel_time": travel_time,
                "average_velocity": average_velocity,
            }
        )


In [ ]:
Kinematics.populate()
Kinematics()


### Table Operators

There are several [operators](https://datajoint.com/docs/core/concepts/query-lang/operators/)
that can help look across or within tables. The most common are:

- `*` for joining tables
- `&` for selecting a subset

`*` helps us look at data across tables.


In [ ]:
Recording * Equipment


`&` helps apply conditions for subsets.


In [ ]:
Kinematics & "average_velocity < 4"


Projection (`proj`) is a useful function for renaming table attributes in a foreign key
reference. For example, a subject entry could also be a referenced as a
[father in a breeding pair](https://github.com/datajoint/element-animal/blob/main/element_animal/genotyping.py#L97-L128).

We can also run simple calculations in projections. Note the use of `CURDATE`, the
current date in native MySQL.


In [ ]:
Subject.proj(age="CURDATE() - dob")

## Exercise

Without referring to the answers below ...

1. Ingest the following subject information and run the kinematics calculations.
2. Visualize the relationship between age and velocity as a scatter plot.


In [ ]:
subjects = [
    [2, "M", "2023-01-03"],
    [3, "F", "2023-01-02"],
    [4, "F", "2023-01-03"],
    [5, "M", "2023-01-05"],
    [6, "M", "2023-01-05"],
    [7, "F", "2023-01-06"],
    [8, "M", "2023-01-07"],
    [9, "F", "2023-01-10"],
]


In [ ]:
from datetime import date
import matplotlib.pyplot as plt

## Your code here ##
a, b = None, None

plt.scatter(a, b)


Our answers hidden in this cell

<!--
## With projection
age, velocity = (
    (Kinematics * Subject)
    .proj(age="CURDATE() - dob", velocity="average_velocity")
    .fetch("age", "velocity")
)
## Without projection
dob, velocity = (Subject * Kinematics).fetch("dob", "average_velocity")
age = np.array([x.days for x in (date.today() - dob)])
plt.scatter(age, velocity)
-->
